In [1]:
!pip install  -q groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 5.5 MB/s eta 0:00:00


In [2]:
import os
import json
from groq import Groq
from typing import List, Dict, Union, Optional

In [3]:
GROQ_API_KEY = "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"

In [4]:
MODEL = 'llama-3.1-8b-instant'
client = Groq(api_key=GROQ_API_KEY)
print("Groq client initialized successfully.")

Groq client initialized successfully.


In [5]:
class ConversationManager:
    """
    Manages a conversation history with summarization and truncation
    using only the Groq API and standard Python.
    """
    def __init__(self, client: Groq, model: str, summarize_every_k: int = 3):
        """
        Initializes the manager.

        Args:
            client: The initialized Groq client.
            model: The model name to use (e.g., 'llama3-8b-8192').
            summarize_every_k: How often to summarize (e.g., every 3 runs).
        """
        self.client = client
        self.model = model
        self.summarize_every_k = summarize_every_k
        self.run_count = 0
        # Start with a base system prompt
        self.history: List[Dict[str, str]] = [
            {"role": "system", "content": "You are a helpful assistant."}
        ]

    def add_message(self, role: str, content: str):
        """Adds a message to the *full* conversation history."""
        self.history.append({"role": role, "content": content})

    def _summarize(self, text_to_summarize: str) -> str:
        """
        Internal method to call the Groq API for summarization.
        """
        if not text_to_summarize:
            return "Nothing to summarize."

        system_prompt = "Summarize the following conversation concisely. Capture the key topics, any questions asked, and the main answers given. This summary will be used as context for a future conversation."

        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": text_to_summarize}
                ],
                temperature=0.1
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"Error during summarization: {e}")
            return "[Summarization failed]"

    def _perform_periodic_summarization(self):
        """
        Checks if it's time to summarize and, if so, replaces the
        conversation history with a summary.
        """
        print(f"\n--- [Attempting Summarization at Run {self.run_count}] ---")

        # We summarize *after* the k-th run is complete
        if self.run_count > 0 and self.run_count % self.summarize_every_k == 0:
            print(f"Condition met (k={self.summarize_every_k}). Summarizing history...")

            # Keep system prompts, but summarize the rest
            system_prompts = [msg for msg in self.history if msg['role'] == 'system']
            conversation_part = [msg for msg in self.history if msg['role'] != 'system']

            if not conversation_part:
                print("No conversation to summarize. Skipping.")
                print("-------------------------------------------------")
                return

            # Format conversation for the summarizer model
            text_to_summarize = "\n".join(
                [f"{m['role']}: {m['content']}" for m in conversation_part]
            )

            print(f"Original History Length (non-system): {len(conversation_part)} messages")

            # Get the summary from the API
            summary = self._summarize(text_to_summarize)

            print(f"New Summary: {summary}")

            # The new history is just the system prompts + the new summary
            self.history = system_prompts + [
                {"role": "system", "content": f"Summary of the conversation so far: {summary}"}
            ]
            print(f"History has been replaced with summary. New history length: {len(self.history)}")

        else:
            print(f"Condition not met (Run {self.run_count} % {self.summarize_every_k} != 0). Skipping summarization.")

        print("-------------------------------------------------")


    def get_truncated_history(self, max_turns: Optional[int] = None, max_chars: Optional[int] = None) -> List[Dict[str, str]]:
        """
        Returns a *copy* of the history, truncated according to the rules.
        It does NOT modify the main `self.history`.
        This is used to prepare the message list for the *next* API call.
        """

        # Always keep system prompts
        system_prompts = [msg for msg in self.history if msg['role'] == 'system']
        conversation_part = [msg for msg in self.history if msg['role'] != 'system']

        # 1. Truncate by number of turns (1 turn = 1 user + 1 assistant)
        if max_turns:
            num_messages_to_keep = max_turns * 2
            # Get the last N messages
            start_index = max(0, len(conversation_part) - num_messages_to_keep)
            conversation_part = conversation_part[start_index:]

        # 2. Truncate by character length
        elif max_chars:
            current_chars = 0
            temp_list = []
            # Iterate in reverse to keep the *latest* messages that fit
            for message in reversed(conversation_part):
                msg_len = len(message['content'])
                if current_chars + msg_len > max_chars:
                    break
                current_chars += msg_len
                temp_list.append(message)
            conversation_part = list(reversed(temp_list)) # Re-reverse to correct order

        # Return the system prompts + the (potentially truncated) conversation
        return system_prompts + conversation_part

    def chat(self, user_message: str, max_turns: Optional[int] = None, max_chars: Optional[int] = None) -> str:
        """
        This is the main function to interact with the chat.
        """
        if self.client is None:
            return "ERROR: Groq client is not initialized. Please check your API key."

        # 1. Add the new user message to the full history
        self.add_message("user", user_message)

        # 2. Get the history to send to the API (which might be truncated)
        messages_to_send = self.get_truncated_history(max_turns=max_turns, max_chars=max_chars)

        # 3. Call Groq API
        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=messages_to_send
            )
            assistant_response = response.choices[0].message.content
        except Exception as e:
            print(f"Error during chat completion: {e}")
            assistant_response = "[API Error]"

        # 4. Add the assistant's response to the full history
        self.add_message("assistant", assistant_response)

        # 5. Increment the run count
        self.run_count += 1

        # 6. Check if it's time to summarize (this happens *after* the run)
        self._perform_periodic_summarization()

        return assistant_response

In [6]:
if client:
    # Initialize the manager to summarize after every 3rd run
    manager = ConversationManager(client, MODEL, summarize_every_k=3)

    print(f"### RUN 1 (k=3) ###")
    response_1 = manager.chat("Hi there! My name is Alex. Can you tell me about the planet Mars?")
    print(f"\nUSER: Hi there! My name is Alex. Can you tell me about the planet Mars?")
    print(f"ASSISTANT: {response_1}")
    print(f"\nFull history length: {len(manager.history)}") # System + User + Assistant = 3

    print("\n" + "="*60 + "\n")

    print(f"### RUN 2 (k=3) ###")
    response_2 = manager.chat("How long would it take to travel there from Earth?")
    print(f"\nUSER: How long would it take to travel there from Earth?")
    print(f"ASSISTANT: {response_2}")
    print(f"\nFull history length: {len(manager.history)}") # 3 + User + Assistant = 5

    print("\n" + "="*60 + "\n")

    print(f"### RUN 3 (k=3) ###")
    # This is the 3rd run, so summarization will trigger *after* this
    response_3 = manager.chat("What about Jupiter's 'Great Red Spot'?")
    print(f"\nUSER: What about Jupiter's 'Great Red Spot'?")
    print(f"ASSISTANT: {response_3}")
    print(f"\nFull history length after summarization: {len(manager.history)}") # Should be 2 (System + New Summary)

    print("\n" + "="*60 + "\n")

    print(f"### RUN 4 (k=3) ###")
    print("Notice: The model will now use the summary as context.")
    # The model should know "Alex" and "Mars" from the summary
    response_4 = manager.chat("Remind me, what was the first planet I asked about?")
    print(f"\nUSER: Remind me, what was the first planet I asked about?")
    print(f"ASSISTANT: {response_4}")
    print(f"\nFull history length: {len(manager.history)}") # 2 + User + Assistant = 4
else:
    print("Client not initialized. Skipping Task 1 Demonstration.")

### RUN 1 (k=3) ###

--- [Attempting Summarization at Run 1] ---
Condition not met (Run 1 % 3 != 0). Skipping summarization.
-------------------------------------------------

USER: Hi there! My name is Alex. Can you tell me about the planet Mars?
ASSISTANT: Hello Alex.  It's great to meet you. Mars, also known as the Red Planet, is the second-smallest planet in our solar system and the fourth planet from the Sun. It's a significant target for exploration due to its proximity to Earth and its potential for supporting past or present life.

Here are some interesting facts about Mars:

1. **Geological History**: Mars is believed to have formed around the same time as Earth, about 4.5 billion years ago. Scientists think that Mars may have had a more Earth-like environment in the past, with flowing rivers, lakes, and even oceans. However, the planet underwent a catastrophic event, possibly due to a massive asteroid impact, which caused widespread destruction and loss of its atmosphere.

2.

In [9]:
if client:
    # Create a new manager just for this demo
    trunc_manager = ConversationManager(client, MODEL, summarize_every_k=999) # k=999 to prevent summarization

    # Manually add a long history
    trunc_manager.add_message("user", "This is message 1. What is 1+1?")
    trunc_manager.add_message("assistant", "1+1 equals 2.")
    trunc_manager.add_message("user", "This is message 3. What is 2+2?")
    trunc_manager.add_message("assistant", "2+2 equals 4. This is a slightly longer response to add characters.")
    trunc_manager.add_message("user", "This is message 5. What is 3+3?")
    trunc_manager.add_message("assistant", "3+3 equals 6. This is the final, most recent message in the history.")

    print(f"Full history created with {len(trunc_manager.history)} messages (1 system + 6 conversation).")
    print("\n" + "="*60 + "\n")

    # --- Demo 1: Limit by max_turns=1 ---
    print("--- Demo: get_truncated_history(max_turns=1) ---")
    # Should return the system prompt + the last 2 messages (1 turn)
    truncated_by_turn = trunc_manager.get_truncated_history(max_turns=1)

    print(f"Truncated history length: {len(truncated_by_turn)}")
    for msg in truncated_by_turn:
        print(msg)

    print("\n" + "="*60 + "\n")

    # --- Demo 2: Limit by max_chars=100 ---
    print("--- Demo: get_truncated_history(max_chars=100) ---")
    # Should return system prompt + as many messages from the END as fit in 100 chars
    truncated_by_char = trunc_manager.get_truncated_history(max_chars=100)

    print(f"Truncated history length: {len(truncated_by_char)}")
    total_chars = 0
    for msg in truncated_by_char:
        if msg['role'] != 'system':
            total_chars += len(msg['content'])
        print(msg)
    print(f"\nTotal characters in conversation: {total_chars} (should be <= 100)")

else:
    print("Client not initialized. Skipping Truncation Demonstration.")

Full history created with 7 messages (1 system + 6 conversation).


--- Demo: get_truncated_history(max_turns=1) ---
Truncated history length: 3
{'role': 'system', 'content': 'You are a helpful assistant.'}
{'role': 'user', 'content': 'This is message 5. What is 3+3?'}
{'role': 'assistant', 'content': '3+3 equals 6. This is the final, most recent message in the history.'}


--- Demo: get_truncated_history(max_chars=100) ---
Truncated history length: 3
{'role': 'system', 'content': 'You are a helpful assistant.'}
{'role': 'user', 'content': 'This is message 5. What is 3+3?'}
{'role': 'assistant', 'content': '3+3 equals 6. This is the final, most recent message in the history.'}

Total characters in conversation: 99 (should be <= 100)


**TASK - 2**

In [10]:
# This is the JSON schema for our classifier tool
# It tells the model *exactly* what structured data to return.
CLASSIFIER_TOOL = [
    {
        "type": "function",
        "function": {
            "name": "classify_chat",
            "description": "Classify the user's message into a category, sentiment, and extract key entities.",
            "parameters": {
                "type": "object",
                "properties": {
                    "category": {
                        "type": "string",
                        "description": "The main topic of the user's message.",
                        "enum": ["support_request", "sales_inquiry", "general_query", "feedback", "other"]
                    },
                    "sentiment": {
                        "type": "string",
                        "description": "The emotional tone of the message.",
                        "enum": ["positive", "neutral", "negative"]
                    },
                    "entities": {
                        "type": "object",
                        "description": "Any extracted entities like product names, emails, or order IDs.",
                        "properties": {
                            "product_name": {"type": "string", "description": "Any mentioned product names."},
                            "user_email": {"type": "string", "description": "The user's email address, if provided."},
                            "order_id": {"type": "string", "description": "Any mentioned order or ticket IDs."}
                        }
                    }
                },
                "required": ["category", "sentiment"]
            }
        }
    }
]

In [15]:
def classify_message(user_message: str) -> Dict[str, Union[str, Dict]]:
    """
    Classifies a single user message using Groq tool calling.
    This version includes safety checks for the tool call response.
    """
    if client is None:
        return {"error": "Groq client not initialized."}

    messages = [
        {"role": "system", "content": "You are a classification assistant. Analyze the user's message and call the `classify_chat` tool with the appropriate category, sentiment, and extracted entities."},
        {"role": "user", "content": user_message}
    ]

    try:
        response = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=CLASSIFIER_TOOL,
            tool_choice={"type": "function", "function": {"name": "classify_chat"}},
            temperature=0.1 # Lower temperature for more deterministic results
        )

        response_message = response.choices[0].message

        # Check if the model decided to call a tool before trying to access it.
        if response_message.tool_calls:
            tool_call = response_message.tool_calls[0]
            if tool_call.function.name == "classify_chat":
                # The arguments are a JSON string, so we must parse them
                arguments_json = tool_call.function.arguments
                structured_data = json.loads(arguments_json)
                return structured_data
            else:
                return {"error": "Model called an unexpected tool."}
        else:
            # This block now handles cases where the model ignored our instruction
            # and returned a plain text response instead.
            error_content = response_message.content or "No text content returned."
            print(f"DEBUG: Model failed to call a tool. It responded with: '{error_content}'")
            return {
                "error": "Model failed to generate a tool call.",
                "model_response": error_content
            }

    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return {"error": str(e)}

In [16]:
if client:
    test_messages = [
        "Hi, my X-1000 rocket boots are broken, can I get a replacement? My email is buzz@light.year and my order was #G-123.",
        "I'm interested in buying the new Llama-3 model. How much is it?",
        "What's the weather like in Tokyo today?",
        "Your new UI is amazing! So much faster and easier to use. Great job!",
        "I hate this new update. It's terrible. My order #T-456 is still missing. - angry_user@web.com"
    ]

    for message in test_messages:
        print(f"--- INPUT ---\n{message}")
        classification = classify_message(message)

        # Pretty-print the JSON output
        print("--- OUTPUT ---")
        print(json.dumps(classification, indent=2))
        print("="*60)

else:
    print("Client not initialized. Skipping Task 2 Demonstration.")

--- INPUT ---
Hi, my X-1000 rocket boots are broken, can I get a replacement? My email is buzz@light.year and my order was #G-123.
--- OUTPUT ---
{
  "category": "support_request",
  "entities": {
    "order_id": "G-123",
    "user_email": "buzz@light.year"
  },
  "sentiment": "negative"
}
--- INPUT ---
I'm interested in buying the new Llama-3 model. How much is it?
--- OUTPUT ---
{
  "category": "sales_inquiry",
  "entities": {
    "product_name": "Llama-3 model"
  },
  "sentiment": "neutral"
}
--- INPUT ---
What's the weather like in Tokyo today?
--- OUTPUT ---
{
  "category": "general_query",
  "entities": {
    "product_name": "Tokyo"
  },
  "sentiment": "neutral"
}
--- INPUT ---
Your new UI is amazing! So much faster and easier to use. Great job!
--- OUTPUT ---
{
  "category": "general_query",
  "entities": {
    "product_name": "UI",
    "user_email": "null"
  },
  "sentiment": "positive"
}
--- INPUT ---
I hate this new update. It's terrible. My order #T-456 is still missing. - a